In [1]:
import pandas as pd

# Cargamos excel
df = pd.read_excel("data_salaries.xlsx")
df.head()

,company,job_description,rating,min_salary,max_salary,median_salary,currency,frequency,city
0,SDG Group,Data Analyst,3.9,25000,27000,26000,EUR,yearly,Madrid
1,Accenture,Data Analyst,3.8,26000,30000,29000,EUR,yearly,Madrid
2,Revolut,Data Analyst,3.9,45000,63000,56000,EUR,yearly,Madrid
3,Santander,Data Analyst,3.8,32000,42000,40000,EUR,yearly,Madrid
4,Minsait,Data Analyst,3.8,25000,30000,27000,EUR,yearly,Madrid


In [2]:
# Limpieza de datos
df = df[df['frequency']=='yearly']
df = df[['city', 'median_salary']].dropna()
quartile_data = []

# Calculamos cuartiles para cada ciudad
for city in ['Madrid', 'Barcelona', 'Berlin']:
    city_salaries = df[df['city'] == city]['median_salary']
    if not city_salaries.empty:
        q25 = city_salaries.quantile(0.25)
        q50 = city_salaries.quantile(0.50)
        q75 = city_salaries.quantile(0.75)
        quartile_data.extend([
            {'Grupo': f'{city}_salary_25', 'Salario': q25, 'Ciudad': city},
            {'Grupo': f'{city}_salary_50', 'Salario': q50, 'Ciudad': city},
            {'Grupo': f'{city}_salary_75', 'Salario': q75, 'Ciudad': city},
        ])

quartiles_df = pd.DataFrame(quartile_data)

In [3]:
def calcular_impuestos_total(salario, ciudad):
    ciudad = ciudad.lower()
    irpf = 0
    ssocial = 0

    # IRPF España
    if ciudad in ['madrid', 'barcelona', 'españa']:
        tramos = [
            (12450, 0.19),
            (20200, 0.24),
            (35200, 0.30),
            (60000, 0.37),
            (float('inf'), 0.45)
        ]
        ssocial = salario * 0.0635

    # IRPF Alemania
    elif ciudad in ['berlin', 'alemania']:
        tramos = [
            (10908, 0.00),
            (61972, 0.30),
            (277825, 0.42),
            (float('inf'), 0.45)
        ]
        ssocial = salario * 0.20

    else:
        return None

    base = 0
    for limite, tipo in tramos:
        if salario > limite:
            irpf += (limite - base) * tipo
            base = limite
        else:
            irpf += (salario - base) * tipo
            break

    total = irpf + ssocial
    return round(total, 2)


In [4]:
quartiles_df['Impuestos'] = quartiles_df.apply(lambda row: calcular_impuestos_total(row['Salario'], row['Ciudad']), axis=1)
# Pasar nominas e impuesto a mensuales
quartiles_df['Salario Bruto Mensual'] = round(quartiles_df['Salario']/12, 2)
quartiles_df['Impuestos'] = round(quartiles_df['Impuestos']/12, 2)
quartiles_df = quartiles_df.drop(columns = ['Salario'])

In [5]:
costes = {
    "Madrid": {
        "Alquiler": 700,
        "Comida": 250,
        "Ocio": 200,
        "Transporte": 30
    },
    "Barcelona": {
        "Alquiler": 700,
        "Comida": 250,
        "Ocio": 200,
        "Transporte": 30
    },
    "Berlin": {
        "Alquiler": 600,
        "Comida": 300,
        "Ocio": 300,
        "Transporte": 50
    }
}
# Añadir columnas según ciudad
quartiles_df["Alquiler"] = quartiles_df["Ciudad"].map(lambda c: costes[c]["Alquiler"])
quartiles_df["Comida"] = quartiles_df["Ciudad"].map(lambda c: costes[c]["Comida"])
quartiles_df["Transporte"] = quartiles_df["Ciudad"].map(lambda c: costes[c]["Transporte"])
quartiles_df["Ocio"] = quartiles_df["Ciudad"].map(lambda c: costes[c]["Ocio"])
quartiles_df["Ahorro"] = round(quartiles_df["Salario Bruto Mensual"] - quartiles_df["Impuestos"] - quartiles_df["Alquiler"] - quartiles_df["Comida"] - quartiles_df["Transporte"] - quartiles_df["Ocio"], 2)

In [6]:
quartiles_df.head()

,Grupo,Ciudad,Impuestos,Salario Bruto Mensual,Alquiler,Comida,Transporte,Ocio,Ahorro
0,Madrid_salary_25,Madrid,755.88,2500.00,700,250,30,200,564.12
1,Madrid_salary_50,Madrid,978.42,3083.33,700,250,30,200,924.91
2,Madrid_salary_75,Madrid,1375.79,4000.00,700,250,30,200,1444.21
3,Barcelona_salary_25,Barcelona,755.88,2500.00,700,250,30,200,564.12
4,Barcelona_salary_50,Barcelona,1086.79,3333.33,700,250,30,200,1066.54


In [7]:
# Preparamos el formato Sankey
sankey_data = []

for _, row in quartiles_df.iterrows():
    Ciudad = row['Ciudad']
    Grupo = row['Grupo']
    Salario = f" {int(row['Salario Bruto Mensual'])}€"

    for expense in ['Impuestos', 'Alquiler', 'Comida', 'Transporte', 'Ocio', 'Ahorro']:
        sankey_data.append({
            'From': Ciudad + Salario,
            'To': expense,
            'Value': row[expense],
            'Grupo': Grupo
        })

sankey_df = pd.DataFrame(sankey_data)


In [8]:
sankey_df.head()

,From,To,Value,Grupo
0,Madrid 2500€,Impuestos,755.88,Madrid_salary_25
1,Madrid 2500€,Alquiler,700.00,Madrid_salary_25
2,Madrid 2500€,Comida,250.00,Madrid_salary_25
3,Madrid 2500€,Transporte,30.00,Madrid_salary_25
4,Madrid 2500€,Ocio,200.00,Madrid_salary_25


In [9]:
# Guardamos el CSV
sankey_df.to_csv("sankey_flourish_ready.csv", index=False)